**Pro**
1. Используем фильм с большим количеством отзывов на сайте Кинопоиск.
Возьмем фильм «Звездные войны: Последние джедаи», т.к. у него больше 100 и плохих, и хороших отзывов.
https://www.kinopoisk.ru/film/718223/


2. Сформируйте 3 отдельных датасета:
  - по 10 хороших и плохих отзывов
  - по 50 хороших и плохих отзывов
  - по 100 хороших и плохих отзывов

3. Сформируйте обучающие наборы данных с помощью `terra-ai-datasets-framework`

4. Подберите архитектуру нейронной сети для классификации собранной базы

5. Создайте таблицу с результатами

In [ ]:
# устанавливаем модуль gdown и обновляем до последней версии
!pip install --upgrade gdown

  Attempting uninstall: gdown
    Found existing installation: gdown 4.7.3
    Uninstalling gdown-4.7.3:
      Successfully uninstalled gdown-4.7.3


In [ ]:
import gdown
import pickle
import os
import shutil

In [ ]:
# скачиваем датасет по ссылке
dataset_url = 'https://storage.yandexcloud.net/aiueducation/Content/DS/L8/dataset.pickle'
gdown.download(dataset_url, quiet=True)


'dataset.pickle'

In [ ]:
# загружаем датасет в переменные со списками
with open('dataset.pickle','rb') as f:
    all_texts_good, all_texts_bad = pickle.load(f)

In [ ]:
print(f'ХОРОШИЕ ОТЗЫВЫ ({len(all_texts_good)} штук):\n')
for t in all_texts_good[:3]:
    print(t)
    print('*'*100)

ХОРОШИЕ ОТЗЫВЫ (158 штук):

В целом, фильм вполне гармонично вписался в ряд с остальными. На наш взгляд, чтобы преисполниться и все стало абсолютно понятно, рекомендуем смотреть с первого эпизода по порядку.
****************************************************************************************************
Я бы даже еще раз сходил, очень хорошо. Потрясающее зрелище.
****************************************************************************************************
6 из 10. Снимаю балы за пресную игру престарелых актёров, утомляющий сценарий, поднадоевшую толерантность и явный коммерческий расчёт. Хотя в целом восприятие фильма позитивное. Вот оно, продолжение которое мы ждали все эти годы. Именно восьмой фильм саги для меня приблизился к планке оригинальной трилогии.
****************************************************************************************************


In [ ]:
print(f'ПЛОХИЕ ОТЗЫВЫ ({len(all_texts_bad)} штук):\n')
for t in all_texts_bad[:3]:
    print(t)
    print('*'*100)

ПЛОХИЕ ОТЗЫВЫ (167 штук):

При неплохой технической стороне и достойной игре актеров из старой гвардии – прекрасной Керри Фишер и не растерявшего свое мастерство Марка Хэмилла, а также более складного образа персонажа Драйвера Кайло Рена, получившийся итог – новое разочарование, которое сыграло на чувстве ностальгии по оригинальной трилогии, вновь выдрало оттуда целые сценарные куски, образовав невнятное «лоскутное одеяло», но так и не стало целостной историей. Новые герои так и не смогли составить конкуренцию классическим персонажам. Довольны, пожалуй, только будут представители мерчендайзингового департамента, чей план продаж будет явно перевыполнен в ближайшие месяцы. Полностью бездушная поделка, продолжающая конвейерный слив всех традиций полюбившихся старых «Звездных войн», безусловно, окупится в прокате и найдет своих фанатов, однако, создаст много трудностей для создания 9 эпизода и пока негласно провозглашает смерть вселенной «Звездных войн» от продюсерской жадности и неумения 

In [ ]:
# Ваше решение

!pip uninstall -y pyarrow
!pip install pyarrow==14.0
!pip uninstall -y pydantic
!pip install pydantic==1.10.13
!pip -q install autokeras==1.0.20
!pip -q install terra-ai-datasets-framework
!pip install --upgrade gdown

# !pip uninstall pyarrow
# !pip install pyarrow==14.0
# !pip uninstall pydantic
# !pip install pydantic==1.10.13
# !pip -q install --upgrade autokeras
# !pip install --upgrade gdown
# !pip install --upgrade terra-ai-datasets-framework

import gdown
import pickle
import os
import shutil
import random
import numpy as np
import pandas as pd
import autokeras as ak
from terra_ai_datasets.create import TextClassification

dataset_url = 'https://storage.yandexcloud.net/aiueducation/Content/DS/L8/dataset.pickle'
gdown.download(dataset_url, quiet=True)

with open('dataset.pickle','rb') as f:
  all_texts_good, all_texts_bad = pickle.load(f)

datasets = [
    (random.sample(all_texts_good[:100], 10), random.sample (all_texts_bad[:100], 10)),
    (random.sample(all_texts_good[:100], 50), random.sample(all_texts_bad[:100], 50)),
    (random.sample(all_texts_good[:100], 100), random.sample(all_texts_bad[:100], 100))
]

models = []

for i, (dataset_good, dataset_bad) in enumerate(datasets):
  print(f"Датасет #{i+1}")
  print("Датасет 'Положительные отзывы':", dataset_good)
  print("Датасет 'Отрицательные отзывы':", dataset_bad)

  def count_total_words(dataset, class_name):
    total_word_count = sum(len(text.split()) for text in dataset)
    return total_word_count

  !mkdir good_{i}
  !mkdir bad_{i}

  with open(f'good_{i}/dataset_good.txt', 'w') as f:
    for text in dataset_good:
      f.write(text + '\n')

  with open(f'bad_{i}/dataset_bad.txt', 'w') as f:
    for text in dataset_bad:
      f.write(text + '\n')

  dataset = TextClassification(
      source_path=[f'/content/good_{i}', f'/content/bad_{i}'],
      max_words_count=1000,
      mode="Length and step",
      preprocessing="None",
      length=10,
      step=2,
      pymorphy=False,
      train_size=0.8,
      one_hot_encoding=True
  )
  dataset.create(use_generator=False)

  # dataset.Y['train']['output_1'] = np.array([[1, 0] if i == 1 else [0, 1] for i in dataset.Y['train']['output_1'][:, 0]])
  # dataset.Y['val']['output_1'] = np.array([[1, 0] if i == 1 else [0, 1] for i in dataset.Y['val']['output_1'][:, 0]])

  text_clf = ak.TextClassifier(
      max_trials=2,
      overwrite=True,
      seed=None,
      num_classes=2,
      multi_label=False,
      metrics=["accuracy"],
      objective="val_accuracy"
  )

text_clf.fit(
    dataset.X['train']['input_1'],
    np.array([y[0] for y in dataset.Y['train']['output_1']]),
    epochs=5,
    batch_size=256,
    validation_data=(dataset.X['val']['input_1'],
                     np.array([y[0] for y in dataset.Y['val']['output_1']])))
model = text_clf.export_model()
model.save(f'model_text_{i}.keras')
models.append(model)

for i, model in enumerate(models):
  print(f"Summary for model_text_{i}:")
  model.summary()
  print("\n" + "="*50 + "\n")

results = []
for i, model in enumerate(models):
  accuracy = model.evaluate(dataset.X['val']['input_1'], np.array([y[0] for y in dataset.Y['val']['output_1']]))[1]
  results.append({
    'Датасет': f'Датасет #{i+1}',
    'Точность модели': accuracy
  })

df = pd.DataFrame(results)
df.set_index('Датасет', inplace=True)
print(df)


Trial 2 Complete [00h 00m 32s]
val_accuracy: 0.8290165066719055

Best val_accuracy So Far: 0.9483895897865295
Total elapsed time: 00h 01m 28s
Epoch 1/5
164/164 [==============================] - 14s 78ms/step - loss: 0.6186 - accuracy: 0.6406 - val_loss: 0.5040 - val_accuracy: 0.7500
Epoch 2/5
164/164 [==============================] - 7s 44ms/step - loss: 0.4002 - accuracy: 0.8150 - val_loss: 0.3710 - val_accuracy: 0.8263
Epoch 3/5
164/164 [==============================] - 8s 46ms/step - loss: 0.2369 - accuracy: 0.9000 - val_loss: 0.2245 - val_accuracy: 0.9059
Epoch 4/5
164/164 [==============================] - 7s 44ms/step - loss: 0.1243 - accuracy: 0.9512 - val_loss: 0.1834 - val_accuracy: 0.9277
Epoch 5/5
164/164 [==============================] - 7s 44ms/step - loss: 0.0786 - accuracy: 0.9713 - val_loss: 0.1539 - val_accuracy: 0.9414
Summary for model_text_0:
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shap

In [ ]:
# Вывод датасета по классам
dataset.visualize()

Класс: good_2


'они хотят как то проявить себя в фильме они же'


Класс: bad_2


'то более оригинальное нежели тупое копирование эпизода 4 смотрится не'

In [ ]:
dataset.summary()

,1_Text,2_Classification
0,стать Джедаем Звучит знакомо да Согласен Но ка...,good_2
1,подкрепляется тупостью Сноука гоблина в красно...,bad_2
2,в этот фильм были вбуханы баснословные пиарили...,bad_2
3,Но не персонажами едиными хороши 'Последние Дж...,good_2
4,и прекрасна выражая апогей его достижений как ...,good_2



Кол-во примеров в train выборке: 41849
Кол-во примеров в val выборке: 10463

Размерность входного массива 1: ()
Размерность выходного массива 1: (2,)
Список классов и количество примеров:


,good_2,bad_2
train,19639,22210
val,4779,5684
